In [2]:
#import of library
#必要なライブラリのインポート
import os
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.preprocessing.image import load_img,img_to_array
import keras.preprocessing.image as Image
from keras.models import Model
from keras.datasets import fashion_mnist
from PIL import Image
import numpy
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import keras
import glob
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import math
import sys
import cv2

In [61]:
#list of variable
#変数リスト
#Division number
#分割数
cv = 10
#research gamma
#調べたいガンマ
#gamma_list = np.logspace(-3, -1, 5)
gamma_list = [0.0025,0.002,0.0015]

#research nu
#gamma_list = [0.0013,0.00131,0.00132,0.00129,0.00128

#調べたいnu
#nu_list = np.logspace(-3,-0.5,5)
#nu = 0.000185

#nuを固定する場合
nu = 0.1

#Number of test data
#実験データの数
data_range = 6000

#その他必要な変数
Tshirt_train = []
shirt_train = []

test = []

train_label = []
test_label = []

divide = data_range//cv

tmp_train = []
tmp_test = []

current_total_f_value = 0

#fashion_mnistを読み込み
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images,test_labels) = fashion_mnist.load_data()

In [56]:
#保存してあるデータをロード
#0 T-シャツ/トップ (T-shirt/top)
#1 ズボン (Trouser)
#2 プルオーバー (Pullover)
#3 ドレス (Dress)
#4 コート (Coat)
#5 サンダル (Sandal)
#6 シャツ (Shirt)
#7 スニーカー (Sneaker)
#8 バッグ (Bag)
#9 アンクルブーツ (Ankle boot)

#train:学習に使うデータ
#test:評価に使うデータ
#noise:学習時に異常データとして混ぜるデータ

#それぞれの特徴抽出の場合でデータを読み込む
#パスは各々で指定する

#HOG特徴抽出後
train = np.load("/home/baba/.keras/datasets/fashion_mnist_hog_pertype/fashion_mnist_hog_0.npy")
test = np.load("/home/baba/.keras/datasets/fashion_mnist_hog_pertype/fashion_mnist_hog_6.npy")
noise = np.load("/home/baba/.keras/datasets/fashion_mnist_hog_pertype/fashion_mnist_hog_1.npy")

#VGG16特徴抽出後
"""
train = np.load("C:/Users/satok/.keras/datasets/fashion_mnist_pertype_feature/fashion_mnist_48_48_0.npy")
test = np.load("C:/Users/satok/.keras/datasets/fashion_mnist_pertype_feature/fashion_mnist_48_48_6.npy")
noise = np.load("C:/Users/satok/.keras/datasets/fashion_mnist_pertype_feature/fashion_mnist_48_48_1.npy")
"""


#特徴抽出前
#train = np.load("C:/Users/satok/.keras/datasets/fashion_mnist_pertype/fashion_mnist_0.npy")
#test = np.load("C:/Users/satok/.keras/datasets/fashion_mnist_pertype/fashion_mnist_6.npy")
#noise = np.load("C:/Users/satok/.keras/datasets/fashion_mnist_pertype/fashion_mnist_1.npy")



#VGG16特徴抽出後にnumpy配列の形を整える
#train = np.reshape(train, (len(train),512))
#test = np.reshape(test, (len(test),512))
#noise = np.reshape(test, (len(noise),512))


#特徴抽出前のデータのnumpy配列の形を整える
"""
test_list = resize_array_images(train_images,48,48,img_array_list)
train = test_list[np.where(train_labels == 0)[0]]
test = test_list[np.where(train_labels == 6)[0]]
noise = test_list[np.where(train_labels == 2)[0]]

train = np.reshape(train, (len(train),2304))
test = np.reshape(test, (len(test),2304))
noise = np.reshape(test, (len(noise),2304))

train = train.astype('float32') / 255.0
test = test.astype('float32') / 255.0
noise = noise.astype('float32') / 255.0
"""

"\ntest_list = resize_array_images(train_images,48,48,img_array_list)\ntrain = test_list[np.where(train_labels == 0)[0]]\ntest = test_list[np.where(train_labels == 6)[0]]\nnoise = test_list[np.where(train_labels == 2)[0]]\n\ntrain = np.reshape(train, (len(train),2304))\ntest = np.reshape(test, (len(test),2304))\nnoise = np.reshape(test, (len(noise),2304))\n\ntrain = train.astype('float32') / 255.0\ntest = test.astype('float32') / 255.0\nnoise = noise.astype('float32') / 255.0\n"

In [97]:
#fashion_mnistをリサイズして返す関数
def resize_array_images(train_images,width,height,img_array_list):
    for i in range(len(train_images)):
        pil_img = Image.fromarray(train_images[i])
        pil_img = pil_img.convert("RGB")
        pil_img = pil_img.resize((width,height), Image.BICUBIC)
        pil_img = pil_img.convert("L")
        
        img_array = numpy.asarray(pil_img)
        img_array_list.append(img_array)
    img_array_list = np.array(img_array_list)
    return img_array_list

In [57]:
#テストデータに加えるノイズの作成
train_noise = noise[0:divide]
test_noise = test[0:divide]

In [62]:
#grid_search and cross_validation
current_total_f_value = 0
best_average_f_value = 0
#グラフ描画用
f_value_list = []

for gamma in gamma_list:
#for nu in nu_list:
        
    for i in range(cv):
        tmp_test.extend(train[i*divide: ((i+1) * divide)])
        tmp_train.extend(train[0:i*divide])
        tmp_train.extend(train[((i+1) * divide) : len(train)])
        
        tmp_train.extend(train_noise)
        tmp_test.extend(test_noise)
        
        clf = svm.OneClassSVM(nu = nu, kernel="rbf", gamma=gamma)
        clf.fit(tmp_train)
        
        pred = clf.predict(tmp_test)
        
        real_true_pred = pred[0:divide]
        real_false_pred = pred[divide : divide * 2]
        
        tp = len(real_true_pred[real_true_pred == 1]) / len(real_true_pred)
        fn = len(real_true_pred[real_true_pred == -1]) / len(real_true_pred)
        
        fp = len(real_false_pred[real_false_pred == -1]) / len(real_false_pred)
        tn = len(real_false_pred[real_false_pred == 1]) / len(real_false_pred)
        
        precision = tp/(tp + fp)
        recall = tp/(tp + fn)
        f_value = (2 * precision * recall) / (precision + recall)
        
        current_total_f_value = current_total_f_value + f_value
        
        tmp_test.clear()
        tmp_train.clear()
        
    current_average_f_value = current_total_f_value / cv
    
    print("gamma : " + str(gamma))
    print("average_f_value : " + str(current_average_f_value) + "\n")
    f_value_list.append(current_average_f_value)
    
    if(best_average_f_value < current_average_f_value):
        best_average_f_value = current_average_f_value
        best_gamma = gamma
    
    current_total_f_value = 0
    current_average_f_value = 0

print("best_gamma : " + str(best_gamma))
print("best_average_value : " + str(best_average_f_value))

gamma : 0.0025
average_f_value : 0.8992997752434777

gamma : 0.002
average_f_value : 0.8994811916613422

gamma : 0.0015
average_f_value : 0.8993275472526487

best_gamma : 0.002
best_average_value : 0.8994811916613422


In [106]:
#グラフを描画したい場合に実行
plt.plot(gamma_list, f_value_list, label = "gamma")
plt.legend()
plt.xlabel("gamma")
#対数軸にしたい場合
plt.xscale("log")
#軸の歯にを指定したい場合
#plt.xlim([0.053,0.057])
#plt.ylim([0.875,0.88])
plt.ylabel("f_value")
plt.title('Graph of searching gamma with nu=0.1')
plt.show

gamma : 0.001
average_acc : 0.0

gamma : 0.0031622776601683794
average_acc : 0.0



KeyboardInterrupt: 

In [63]:
#パラメータを決めた後、評価に使うテストデータを読み込む
test_train = np.load("/home/baba/.keras/datasets/fashion_mnist_hog_pertype_test/fashion_mnist_hog_test_0.npy")
test_test = np.load("/home/baba/.keras/datasets/fashion_mnist_hog_pertype_test/fashion_mnist_hog_test_6.npy")

test_train = np.reshape(test_train, (len(test_train),900))
test_test = np.reshape(test_test, (len(test_test),900))

In [64]:
#テストデータに加えるノイズの作成
#ここのノイズデータは上のほうで使ったnoiseを使用
train_noise = noise[0:divide]
train_noise.shape

In [65]:
#学習用データの作成
tmp_train = []
tmp_train.extend(train)
tmp_train.extend(train_noise)
#評価用データの作成
tmp_test_train = []
tmp_test_train.extend(test_train)
tmp_test_train.extend(test_test)

In [67]:
#svmのモデルを作成
svm_model = svm.OneClassSVM(nu = 0.1, kernel="rbf",gamma=0.002)
#識別関数を計算
svm_model.fit(tmp_train)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.002, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [69]:
#分類、評価を行う
pred = svm_model.predict(tmp_test_train)
        
real_true_pred = pred[0:1000]
real_false_pred = pred[1000 : 2000]
        
tp = len(real_true_pred[real_true_pred == 1]) / len(real_true_pred)
fn = len(real_true_pred[real_true_pred == -1]) / len(real_true_pred)
        
fp = len(real_false_pred[real_false_pred == -1]) / len(real_false_pred)
tn = len(real_false_pred[real_false_pred == 1]) / len(real_false_pred)
        
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f_value = (2 * precision * recall) / (precision + recall)

print("f_value " + str(f_value))

f_value 0.9021361152508693


In [43]:
#以下,HOG特徴量で画像を保存する場合

#ファイル内のjpg画像を一括読み込みする関数
def fileRead():
    data = []
    for file in glob.glob("/home/baba/.keras/datasets/fashion-mnist-test-images/*.jpg"):
        data.append( cv2.imread(file, 0) )
    return data

In [ ]:
#ファイルから画像を読み込む準備
img = cv2.imread("/home/baba/.keras/datasets/fashion-mnist-images/fashion_mnist_BICUBIC00000.jpg",0)
cv2.imshow("image",img)
cv2.waitKey()

In [44]:
#読み込み
data = fileRead()

In [45]:
#HOGのパラメータ決定
#基本的にはこれでいい
#winsizeは画像のサイズ
hog_train = []
winsize = (48,48)
block_size = (16,16)
block_stride = (8,8)
cell_size = (8,8)
nbins = 9

hog = cv2.HOGDescriptor(winsize,block_size,block_stride,cell_size,nbins)
for filepath in data:
    hog_train.append(hog.compute(filepath))

hog_train = np.array(hog_train)

In [46]:
#hog_train.shape
hog_train.shape

(10000, 900, 1)

In [47]:
#svmで分類できるように形を整える
hog_train = np.reshape(hog_train, (len(hog_train),900))

In [48]:
hog_train.shape

(10000, 900)

In [49]:

train = hog_train[np.where(test_labels == 0)[0]]
test = hog_train[np.where(test_labels == 6)[0]]
noise = hog_train[np.where(test_labels == 1)[0]]

In [53]:
np.save(os.path.join("/home/baba/.keras/datasets/fashion_mnist_hog_pertype_test/" , "fashion_mnist_hog_test_1.npy"),noise)